# Lab 5. Обработка текста - исповедь светлокожего вдовца



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from collections import Counter

In [2]:
with open('lolita.txt') as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

drop_not_letters = lambda s: ''.join(list(filter(lambda c: c.isalpha() or c == ' ' or c == '.' or c == ',', s)))
text_sample = ''.join(filter(drop_not_letters, text_sample))
text_sample = text_sample.lower()

In [3]:
text_sample

' foreword              lolita, or the confession of a white widowed male, such were the two titles under which the writer of the present note received the strange pages it preambulates. humbert humbert, their author, had died in legal captivity, of coronary thrombosis, on november , , a few days before his trial was scheduled to start. his lawyer, my good friend and relation, clarence choate clark, esq., now of he district of columbia bar, in asking me to edit the manuscript, based his request on a clause in his clients will which empowered my eminent cousin to use the discretion in all matters pertaining to the preparation of lolita for print. mr. clarks decision may have been influenced by the fact that the editor of his choice had just been awarded the poling prize for a modest work do the senses make sense wherein certain morbid states and perversions had been discussed.       my task proved simpler than either of us had anticipated. save for the correction of obvious solecisms an

Произведение содержит в себе введение, которое написано в другом стиле, чем сама книга. Так же нужно удалить слова 'chapter'

In [4]:
index = text_sample.find('part one')
text_sample = text_sample[index:]
text_sample = text_sample.replace('part one', '')
text_sample = text_sample.replace('part two', '')

In [5]:
text_sample

'                                             lolita, light of my life, fire of my loins. my sin, my soul. loleeta the tip of the tongue taking a trip of three steps down the palate to tap, at three, on the teeth. lo. lee. ta.         she was lo, plain lo, in the morning, standing four feet ten in one sock. she was lola in slacks. she was dolly at school. she was dolores on the dotted line. but in my arms she was always lolita.         did she have a precursor she did, indeed she did. in point of fact, there might have been no lolita at all had i not loved, one summer, a certain initial girlchild. in a princedom by the sea. oh when about as many years before lolita was born as my age was that summer. you can always count on a murderer for a fancy prose style.         ladies and gentlemen of the jury, exhibit number one is what the seraphs, the misinformed, simple, noblewinged seraphs, envied. look at this tangle of thorns.                                             i was born in , in 

Преобразуем текст в последовательность чисел.

In [6]:
def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])

    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

Сгенерируем train и test batch, будем предсказывать конец по началу.

In [7]:
sequence_length = 512
batch_size = 32

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(batch_size):
        batch_start = np.random.randint(0, len(sequence) - sequence_length)
        chunk = sequence[batch_start: batch_start + sequence_length]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

Напишем функцию для генерации текста, использующюю модель.

In [15]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' .', prediction_len=200, temp=0.35):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text

    _, hidden = model(train, hidden)

    inp = train[-1].view(-1, 1, 1)

    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char

    return predicted_text

Напишем класс для модели. Сначала преобразуем символы в вектор. Передадим векторы в LSTM слой.

In [16]:
class TextRNN(nn.Module):

    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)

    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)

    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

Обучим нашу сеть.



1.   Используем ADAM в качестве оптимизатора
2.   Используем шедулер, уменьшающий learning rate на плато.
3.   Будем обучать модель на протяжении 50000 эпох



In [17]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=5,
    verbose=True,
    factor=0.5
)

n_epochs = 50000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(batch_size)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 2.403441905975342
 .                                                                                                                                                                                                        
Loss: 1.8566676664352417
 .                                                                                                                                                                                                        
Loss: 1.7226547694206238
 .                                                                                                                                                                                                        
Loss: 1.6456644988059999
 .                                                                                                                                                                                                        
Loss: 1.596467752456665
 . i had in the was the store she was a still the little started 

KeyboardInterrupt: 

In [18]:
model.eval()

print(evaluate(
    model,
    char_to_idx,
    idx_to_char
    )
)

 . i saw the local miss opposite were coming to the sofa i said invented a couple of spare back of the stairs. i had forever the arm of a laprobe of the landing that she had still in the cold slang of i
